In [3]:
!pip install bs4

  Using cached bs4-0.0.2-py2.py3-none-any.whl.metadata (411 bytes)
Using cached bs4-0.0.2-py2.py3-none-any.whl (1.2 kB)


In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [17]:
url = "http://quotes.toscrape.com/"
response=requests.get(url)

In [20]:
!pip install lxml

In [21]:
soup=BeautifulSoup(response.text,"lxml")

In [31]:
quotes=soup.find_all('div',class_='quote')
quotes

[<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
 <span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
 <span>by <small class="author" itemprop="author">Albert Einstein</small>
 <a href="/author/Albert-Einstein">(about)</a>
 </span>
 <div class="tags">
             Tags:
             <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
 <a class="tag" href="/tag/change/page/1/">change</a>
 <a class="tag" href="/tag/deep-thoughts/page/1/">deep-thoughts</a>
 <a class="tag" href="/tag/thinking/page/1/">thinking</a>
 <a class="tag" href="/tag/world/page/1/">world</a>
 </div>
 </div>,
 <div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
 <span class="text" itemprop="text">“It is our choices, Harry, that show what we truly are, far more than our abilities.”</span>
 <span>by <small class="author" itempr

In [25]:
data=[]

In [26]:
for quote in quotes:
    text = quote.find("span", class_="text").text
    author = quote.find("small", class_="author").text
    tags = [tag.text for tag in quote.find_all("a", class_="tag")]

    data.append({
        "Quote": text,
        "Author": author,
        "Tags": ", ".join(tags)
    })

In [27]:
data

[{'Quote': '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”',
  'Author': 'Albert Einstein',
  'Tags': 'change, deep-thoughts, thinking, world'},
 {'Quote': '“It is our choices, Harry, that show what we truly are, far more than our abilities.”',
  'Author': 'J.K. Rowling',
  'Tags': 'abilities, choices'},
 {'Quote': '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”',
  'Author': 'Albert Einstein',
  'Tags': 'inspirational, life, live, miracle, miracles'},
 {'Quote': '“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”',
  'Author': 'Jane Austen',
  'Tags': 'aliteracy, books, classic, humor'},
 {'Quote': "“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”",
  'Author': 'Marilyn Monroe',
  'Tags': 'be-yourself, inspirational'},

In [29]:
df = pd.DataFrame(data)

# Show DataFrame
df

,Quote,Author,Tags
0,“The world as we have created it is a process ...,Albert Einstein,"change, deep-thoughts, thinking, world"
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"abilities, choices"
2,“There are only two ways to live your life. On...,Albert Einstein,"inspirational, life, live, miracle, miracles"
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"aliteracy, books, classic, humor"
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"be-yourself, inspirational"
5,“Try not to become a man of success. Rather be...,Albert Einstein,"adulthood, success, value"
6,“It is better to be hated for what you are tha...,André Gide,"life, love"
7,"“I have not failed. I've just found 10,000 way...",Thomas A. Edison,"edison, failure, inspirational, paraphrased"
8,“A woman is like a tea bag; you never know how...,Eleanor Roosevelt,misattributed-eleanor-roosevelt
9,"“A day without sunshine is like, you know, nig...",Steve Martin,"humor, obvious, simile"


In [32]:
import time

In [33]:
base_url = "http://books.toscrape.com/catalogue/page-{}.html"

In [39]:
all_books=[]

In [40]:
for page in range(1,6):
    print(f"Scraping Page {page}...")
    url=base_url.format(page)
    response=requests.get(url)

    if response.status_code!=200:
        print('failed to retrieve page')
        continue
    soup=BeautifulSoup(response.text,'lxml')

    books=soup.find_all('article',class_='product_pod')


    for book in books:
        title=book.h3.a['title']
        price = book.find("p", class_="price_color").text
        price = float(price.replace("£", "").replace("Â", "").strip())

        rating_class = book.find("p", class_="star-rating")["class"][1]
        rating_map = {
            "One": 1,
            "Two": 2,
            "Three": 3,
            "Four": 4,
            "Five": 5
        }
        rating = rating_map.get(rating_class, 0)

        availability = book.find("p", class_="instock availability").text.strip()

        relative_link = book.h3.a["href"]
        product_url = "http://books.toscrape.com/catalogue/" + relative_link

        all_books.append({
            "Title": title,
            "Price": price,
            "Rating": rating,
            "Availability": availability,
            "Product_URL": product_url
        })

        time.sleep(1)

Scraping Page 1...
Scraping Page 2...
Scraping Page 3...
Scraping Page 4...
Scraping Page 5...


In [41]:
df = pd.DataFrame(all_books)

In [42]:
df

,Title,Price,Rating,Availability,Product_URL
0,A Light in the Attic,51.77,3,In stock,http://books.toscrape.com/catalogue/a-light-in...
1,Tipping the Velvet,53.74,1,In stock,http://books.toscrape.com/catalogue/tipping-th...
2,Soumission,50.10,1,In stock,http://books.toscrape.com/catalogue/soumission...
3,Sharp Objects,47.82,4,In stock,http://books.toscrape.com/catalogue/sharp-obje...
4,Sapiens: A Brief History of Humankind,54.23,5,In stock,http://books.toscrape.com/catalogue/sapiens-a-...
...,...,...,...,...,...
95,Lumberjanes Vol. 3: A Terrible Plan (Lumberjan...,19.92,2,In stock,http://books.toscrape.com/catalogue/lumberjane...
96,"Layered: Baking, Building, and Styling Spectac...",40.11,1,In stock,http://books.toscrape.com/catalogue/layered-ba...
97,Judo: Seven Steps to Black Belt (an Introducto...,53.90,2,In stock,http://books.toscrape.com/catalogue/judo-seven...
98,Join,35.67,5,In stock,http://books.toscrape.com/catalogue/join_902/i...


In [43]:
print("\nShape:", df.shape)


Shape: (100, 5)


In [44]:
df.to_csv("books_data.csv", index=False)